In [4]:
'''
- finish grid search 


- add dedupe by datamade 
- think about graph based methods
- lsi
http://blog.josephwilk.net/projects/latent-semantic-analysis-in-python.html
https://radimrehurek.com/gensim/dist_lsi.html
- add the DBSCAN bit
- lookup norwigs thing about how properly design a class
- lsh blog
http://matthewcasperson.blogspot.com/2013/11/minhash-for-dummies.html
https://cran.r-project.org/web/packages/textreuse/vignettes/textreuse-minhash.html
http://stackoverflow.com/questions/19701052/how-many-hash-functions-are-required-in-a-minhash-algorithm/19711615#19711615
'''

'\n- finish grid search \n\n\n- add dedupe by datamade \n- think about graph based methods\n- lsi\nhttp://blog.josephwilk.net/projects/latent-semantic-analysis-in-python.html\nhttps://radimrehurek.com/gensim/dist_lsi.html\n- add the DBSCAN bit\n- lookup norwigs thing about how properly design a class\n- lsh blog\nhttp://matthewcasperson.blogspot.com/2013/11/minhash-for-dummies.html\nhttps://cran.r-project.org/web/packages/textreuse/vignettes/textreuse-minhash.html\nhttp://stackoverflow.com/questions/19701052/how-many-hash-functions-are-required-in-a-minhash-algorithm/19711615#19711615\n'

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [110]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# need to add a class for graph theory
# nn class
class Deduper_NN(object):
    '''
    I need to re-evaluate whether or not I want the state of the model/vector space 
    being saved in the event that I dont' I should just kill the self.model = model.fit() stuff
    and pass parameters from one function to another.
    
    methods
    ------
    train
        - model type
    predict
    preprocess
        - various stuff
    '''

    metrics = [
        'cosine', 
        'euclidean',
        'dice', 
        'jaccard', 
        'braycurtis',
        'canberra', 
    ]
    
    vector_space = None

    def read_in_the_file(self, file_name):
        
        #read in subject file
        with open(file_name) as f:
             self.orig_file = [line.strip() for line in f]
    
    def build_vectorizer(self, corpus, model_type='bag of words', ngrams=1, tokenizer='char'):
        '''
        *add word2vec
        '''
        
        #think of params
        params = {
            'analyzer': tokenizer,
            'ngram_range' : (1, ngrams)
        }
    
        if model_type == 'bag of words':
            vectorizer = CountVectorizer(**params)
        elif model_type ==  'tfidf':
            vectorizer = TfidfVectorizer(**params)
        
        self.vector_space = vectorizer.fit_transform(corpus) 
        self.vectorizer = vectorizer 
    
    def find_all_duplicates(self):
        
        #find all duplicates
        all_dups_dict = {idx : self.predict(line) for idx, line in enumerate(self.orig_file)}
        return all_dups_dict
    
    def fit_model(self, model_type='brute', vector_space_params={}):
        '''
        fits model operating under the assumption that there's a model already built
        '''

        if model_type == 'brute':
            self.model = NearestNeighbors(algorithm='brute')
        elif model_type == 'lsh':
            self.model = LSHForest()
        elif model_type == 'annoy':
            self.model = Annoy()

        self.model.fit(self.vector_space)
        print self.model        

    def predict(self, new_data_pt, radius_threshold=.25):
        '''
        not sure how to find the optimal threshold here
        '''
        #careful to note that it takes a single string and converts to a list object of strings
        pt = self.vectorizer.transform([new_data_pt])
        
        #how to find optimal radius?
        distance_from_origin, indices = self.model.radius_neighbors(pt, radius=radius_threshold)
        
        #unpacking
        distance_from_origin = distance_from_origin[0]
        indices = indices[0]

        grabbing_the_lines_from_file = [self.orig_file[index] for index in indices]

        return grabbing_the_lines_from_file
    
    def grid_search(self):
        '''
        I: target string
        O: prints all combinations of comparisons
        
        * this goes in the master deduper class
        '''
        
        #preprocessing variables
            #spaces or no spaces
            #combinations there of.
        
        vector_space_params = {
            'model_types' : ['bag of words', 'tfidf'], #add lsi and word2vec
            'ngrams' : [1,2,3,4],
            'tokenizer' : ['char', 'word'],
            #or some combination there of, to do this we need to output and concat
        }
        
        
        
        #fit the vector-space
        #char-grams, words
            #unagrams, bigrams, tri-grams
            
        
        #model selection
        model_params = {
            'model_type' : ['brute', 'annoy', 'lsh']
            #fill the rest in later
        }
        
        #distances
        metrics = [
            'cosine', 
            'euclidean',
            'dice', 
            'jaccard', 
            'braycurtis',
            'canberra', 
        ]
        
        model_params['metrics'] = metric
        
        all_params = {
            'preprocessing': None,
            'vector_space': vector_space_params,
            'nn_algo': model_params,
        }
        
        for nn_algo in all_params['nn_algo']['model_type']:
            for vector_space_model in all_params['vector_space']['model_types']:
                for gram in  all_params['vector_space']['ngrams']:
                    for type_of_tokenizer in  all_params['vector_space']['tokenizer']:
                        for metric in metrics:
                            params = {
                                nn_algo,
                                vector_space_model,
                                type_of_tokenizer,
                                metric
                            }
                            deduper.train()
                            #print to out 
                
        
        #how do you gauge the quality of matches?
        
        pass
    
    #since this isn't a nn search model it belongs in the biggest deduper
    def brute_force_deduper(self, list_of_strings, comparison_algo, threshold=None):
        '''
        I: self explanatory
        O: dictionary {string: sorted list of matches}
        '''
        big_bag = {}
        #to deep copy or not to deep copy

        for index, s1 in enumerate(list_of_strings):
            small_bag = get_all_comparisons(list_of_strings[index:], comparison_algo)
            big_bag[s1] = sorted(small_bag, key=lambda x: x[0], reverse=True)

        return big_bag
    
    def make_hist(self):
        #use a numpy array since the size is already pre-defined
        hist_bag = []
        
        for index in self.vector_space.indices:

            observation = self.vector_space[index]

            dist, idx = self.model.kneighbors(observation, n_neighbors=2)
            dist, idx = dist[0], idx[0]

            #find out which position the current index is in
            remove_this_arg = [k for k, i in enumerate(idx) if i == index]
            dist = [i for k, i in enumerate(dist) if i != remove_this_arg[0]]
            
            hist_bag.append(dist)
                
        return hist_bag


if __name__ == '__main__':
    deduper = Deduper_NN()
    deduper.read_in_the_file('target.txt')
    deduper.build_vectorizer(deduper.orig_file)
    deduper.fit_model()
    histo = deduper.make_hist()

    for i in range(10):
        print '=' * 50
        print '\n'.join(deduper.predict(deduper.orig_file[i], 3))


NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=4, n_neighbors=5, p=2, radius=1.0)


ImportError: [joblib] Attempting to do parallel computing without protecting your import on a system that does not support forking. To use parallel-computing in a script, you must protect your main loop using "if __name__ == '__main__'". Please see the joblib documentation on Parallel for more information

In [102]:
if __name__ == '__main__':
    import numpy as np
    new_data_pt = 'have a great day gre'
    #careful to note that it takes a single string and converts to a list object of strings

    
    hist_bag = []
    for index in deduper.vector_space.indices:

        observation = deduper.vector_space[index]

        dist, idx = deduper.model.kneighbors(observation, n_neighbors=1)
        dist, idx = dist[0], idx[0]

#         #find out which position the current index is in
#         remove_this_arg = [k for k, i in enumerate(idx) if i == index]
#         dist = [i for k, i in enumerate(dist) if i != remove_this_arg[0]]

#         hist_bag.append(dist)

        break

    #instead of naively removing the first one lets just remove the one with the same index
    
    

#how to find optimal radius?
# distance_from_origin, indices = model.kneighbors(pt, radius=radius_threshold)



ImportError: [joblib] Attempting to do parallel computing without protecting your import on a system that does not support forking. To use parallel-computing in a script, you must protect your main loop using "if __name__ == '__main__'". Please see the joblib documentation on Parallel for more information

In [ ]:
def histogram_maker(model):
    # loop through all the points in the nearest neighbor algo
    # return a histogram, prints time it took to loop through all the points
    
    

In [107]:
def get_all_comparisons(main_str, strings, comparison_algo, threshold=None):
    '''
    I: string, list of strings, string comparison algo eg. levenshtien, threshold
    O: list of tuples (match score, weight) 
    
    Takes a target string and compares it to the rest of strings in the list
    USE --

    get_all_comparisons('check', ['check1'], fuzz.ratio) 
    >>> [(91, 'check1')]
    '''
    match_bag = []

    for str_ in strings:
        match_rating = comparison_algo(main_str, str_)

        if threshold:
            if match_rating > threshold:
                match_bag.append((match_rating, str_))
        else:
            match_bag.append((match_rating, str_))

    return match_bag

from fuzzywuzzy import fuzz

get_all_comparisons('check', ['check1'], fuzz.ratio) 

[(91, 'check1')]